# Advent of Code 2024

In [ ]:
from aocd.models import Puzzle
from pathlib import Path
puzzle = Puzzle(year=2024, day=int(Path(__vsc_ipynb_file__).stem))
print(puzzle.url)
print(puzzle.input_data)


# Part 1

In [16]:
example = """x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02"""

In [21]:
from aocd.models import Puzzle
from pathlib import Path
import functools
puzzle = Puzzle(year=2024, day=int(Path(__vsc_ipynb_file__).stem))

def solve_a(input_data):
    initial_values, gates_data = input_data.split('\n\n')

    # Initialize tree with initial values
    tree = {}
    for line in initial_values.splitlines():
        wire_name, value = line.split(': ')
        tree[wire_name] = int(value)

    # Build the expression tree
    for line in gates_data.splitlines():
        inputs, output = line.split(' -> ')
        input1, op, input2 = inputs.split()
        tree[output] = (op, input1, input2)

    @functools.cache
    def evaluate(wire_name, depth=0):
        indent = "  " * depth
        node = tree[wire_name]
        if isinstance(node, int):
            return node
        else:
            op, operand1, operand2 = node
            value1 = evaluate(operand1, depth + 1)
            value2 = evaluate(operand2, depth + 1)
            if op == 'AND':
                result = value1 & value2
            elif op == 'OR':
                result = value1 | value2
            elif op == 'XOR':
                result = value1 ^ value2
            return result

    # Evaluate output wires
    z_wires = sorted([wire for wire in tree if wire.startswith('z')], key=lambda x: int(x[1:]))
    binary_string = ''.join(str(evaluate(z_wire)) for z_wire in z_wires)
    
    reversed_binary_string = binary_string[::-1]

    # Convert to decimal
    return int(reversed_binary_string, 2)

assert(solve_a(example) == 4)
puzzle.answer_a = solve_a(puzzle.input_data)

# Part 2

The binary numbers are ordered so that the x00 is the least significant bit.

In [22]:
from itertools import combinations, product

def s(w, v):
    w.update(v)

def e(g, w):
    o = []
    for a, op, b, _, c in g:
        if a in w and b in w:
            if op == 'AND':
                w[c] = w[a] & w[b]
            elif op == 'OR':
                w[c] = w[a] | w[b]
            elif op == 'XOR':
                w[c] = w[a] ^ w[b]
            o.append((a, op, b, _, c))
    for x in o:
        g.remove(x)

def p_a(d):
    i, g = d.split('\n\n')
    w = {}
    for l in i.splitlines():
        n, v = l.split(': ')
        w[n] = int(v)
    g = [tuple(l.split()) for l in g.splitlines()]
    while g:
        e(g, w)
    z = [w[f'z{i:02}'] for i in range(len([k for k in w if k.startswith('z')]))]
    return int(''.join(map(str, z[::-1])), 2)

puzzle.answer_a = p_a(puzzle.input_data)


In [ ]:

def p_b(d):
    i, ops = d.split('\n\n')
    w_init = {}
    for l in i.splitlines():
        n, v = l.split(': ')
        w_init[n] = int(v)
    ops = [tuple(l.split()) for l in ops.splitlines()]

    x_bits = len([k for k in w_init if k.startswith('x')])
    y_bits = len([k for k in w_init if k.startswith('y')])

    def simulate(w_init, ops, x_val, y_val):
        w = w_init.copy()
        g = ops[:]
        for i in range(x_bits):
            w[f'x{i:02}'] = (x_val >> i) & 1
        for i in range(y_bits):
            w[f'y{i:02}'] = (y_val >> i) & 1
        while g:
            e(g, w)
        z_bits = len([k for k in w if k.startswith('z')])
        z_val = 0
        for i in range(z_bits):
            z_val |= (w.get(f'z{i:02}', 0) << i)
        return z_val

    diff_gates = set()
    for x_val, y_val in product(range(2**x_bits), range(2**y_bits)):
        expected_z = x_val + y_val
        simulated_z = simulate(w_init, ops, x_val, y_val)
        for i in range(max(x_bits, y_bits) + 1):
            if (expected_z >> i) & 1 != (simulated_z >> i) & 1:
                for op in ops:
                    if op[4] == f'z{i:02}':
                        diff_gates.add(op)

    for swapped_gates in combinations(combinations(diff_gates, 2), 4):
        swapped_ops = ops[:]
        swapped_wire_names = set()

        for pair in swapped_gates:
          a,b = pair
          swapped_wire_names.add(a[4])
          swapped_wire_names.add(b[4])
          ia = swapped_ops.index(a)
          ib = swapped_ops.index(b)
          a_op, b_op = list(swapped_ops[ia]), list(swapped_ops[ib])
          a_op[4], b_op[4] = b_op[4], a_op[4]
          swapped_ops[ia] = tuple(a_op)
          swapped_ops[ib] = tuple(b_op)

        all_correct = True
        for x_val, y_val in product(range(2**x_bits), range(2**y_bits)):
            expected_z = x_val + y_val
            simulated_z = simulate(w_init, swapped_ops, x_val, y_val)
            if expected_z != simulated_z:
                all_correct = False
                break

        if all_correct:
            swapped_wires = sorted(swapped_wire_names)
            return ','.join(swapped_wires)

puzzle.answer_b = p_b(puzzle.input_data)

In [ ]:
from itertools import combinations

def s(w, v):
    w.update(v)

def e(g, w):
    o = []
    for a, op, b, _, c in g:
        if a in w and b in w:
            if op == 'AND':
                w[c] = w[a] & w[b]
            elif op == 'OR':
                w[c] = w[a] | w[b]
            elif op == 'XOR':
                w[c] = w[a] ^ w[b]
            o.append((a, op, b, _, c))
    for x in o:
        g.remove(x)

def p_a(d):
    i, g = d.split('\n\n')
    w = {}
    for l in i.splitlines():
        n, v = l.split(': ')
        w[n] = int(v)
    g = [tuple(l.split()) for l in g.splitlines()]
    while g:
        e(g, w)
    z = [w[f'z{i:02}'] for i in range(len([k for k in w if k.startswith('z')]))]
    return int(''.join(map(str, z[::-1])), 2)

puzzle.answer_a = p_a(puzzle.input_data)

def p_b(d):
    i, ops = d.split('\n\n')
    w_init = {}
    for l in i.splitlines():
        n, v = l.split(': ')
        w_init[n] = int(v)
    ops = [tuple(l.split()) for l in ops.splitlines()]

    x_bits = len([k for k in w_init if k.startswith('x')])
    y_bits = len([k for k in w_init if k.startswith('y')])

    def simulate(w_init, ops, x_val, y_val):
        w = w_init.copy()
        g = ops[:]
        for i in range(x_bits):
            w[f'x{i:02}'] = (x_val >> i) & 1
        for i in range(y_bits):
            w[f'y{i:02}'] = (y_val >> i) & 1
        while g:
            e(g, w)
        z_bits = len([k for k in w if k.startswith('z')])
        z_val = 0
        for i in range(z_bits):
            z_val |= (w.get(f'z{i:02}', 0) << i)
        return z_val

    diff_gates = set()
    
    # Iterate through a smaller set of input combinations for initial gate identification
    for x_val in range(2 ** min(x_bits, 5)): # Limit to 5 bits or the number of x_bits
        for y_val in range(2 ** min(y_bits, 5)):
            expected_z = x_val + y_val
            simulated_z = simulate(w_init, ops, x_val, y_val)
            for i in range(max(x_bits, y_bits) + 1):
                if (expected_z >> i) & 1 != (simulated_z >> i) & 1:
                    for op in ops:
                        if op[4] == f'z{i:02}':
                            diff_gates.add(op)

    for swapped_gates in combinations(combinations(diff_gates, 2), 4):
        swapped_ops = ops[:]
        swapped_wire_names = set()
        for pair in swapped_gates:
            a, b = pair
            swapped_wire_names.add(a[4])
            swapped_wire_names.add(b[4])
            ia = swapped_ops.index(a)
            ib = swapped_ops.index(b)
            a_op, b_op = list(swapped_ops[ia]), list(swapped_ops[ib])
            a_op[4], b_op[4] = b_op[4], a_op[4]
            swapped_ops[ia] = tuple(a_op)
            swapped_ops[ib] = tuple(b_op)

        all_correct = True
        # Test all input combinations only after potential swaps are made
        for x_val in range(2**x_bits):
            for y_val in range(2**y_bits):
                expected_z = x_val + y_val
                simulated_z = simulate(w_init, swapped_ops, x_val, y_val)
                if expected_z != simulated_z:
                    all_correct = False
                    break
            if not all_correct:
                break
        if all_correct:
            swapped_wires = sorted(swapped_wire_names)
            return ','.join(swapped_wires)

puzzle.answer_b = p_b(puzzle.input_data)

In [ ]:
def analyze_input(input_data):
    """
    Analyzes the puzzle input data and prints information about the circuit.

    Args:
        input_data: The raw puzzle input string.
    """
    initial_values_str, gates_str = input_data.split('\n\n')

    initial_values = {}
    for line in initial_values_str.splitlines():
        wire_name, value = line.split(': ')
        initial_values[wire_name] = int(value)

    x_bits = len([k for k in initial_values if k.startswith('x')])
    y_bits = len([k for k in initial_values if k.startswith('y')])

    gates = [tuple(line.split()) for line in gates_str.splitlines()]

    z_bits = len(set(op[4] for op in gates if op[4].startswith('z')))

    num_and_gates = sum(1 for op in gates if op[1] == 'AND')
    num_or_gates = sum(1 for op in gates if op[1] == 'OR')
    num_xor_gates = sum(1 for op in gates if op[1] == 'XOR')

    print(f"Number of x bits: {x_bits}")
    print(f"Number of y bits: {y_bits}")
    print(f"Number of z bits: {z_bits}")
    print(f"Total number of logic gates: {len(gates)}")
    print(f"  Number of AND gates: {num_and_gates}")
    print(f"  Number of OR gates: {num_or_gates}")
    print(f"  Number of XOR gates: {num_xor_gates}")

analyze_input(puzzle.input_data)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb

def visualize_circuit(input_data):
    """
    Visualizes the circuit described in the puzzle input data as a graph.

    Args:
        input_data: The raw puzzle input string.
    """
    _, gates_str = input_data.split('\n\n')
    gates = [tuple(line.split()) for line in gates_str.splitlines()]

    G = nx.DiGraph()
    node_colors = {}

    for a, op, b, _, c in gates:
        G.add_edge(a, c, operation=op)
        G.add_edge(b, c, operation=op)
        if a.startswith('x'):
            node_colors[a] = 'lightblue'
        elif a.startswith('y'):
            node_colors[a] = 'lightgreen'
        if b.startswith('x'):
            node_colors[b] = 'lightblue'
        elif b.startswith('y'):
            node_colors[b] = 'lightgreen'
        
        if c.startswith('z'):
            node_colors[c] = 'orange'
        else:
            node_colors[c] = 'pink'

        if op == 'AND':
          op_color = 'red'
        elif op == 'OR':
          op_color = 'green'
        elif op == 'XOR':
          op_color = 'blue'
        else:
          op_color = 'gray'

        # Darken the node color based on operation type
        base_color = to_rgb(node_colors.get(c, 'pink'))  # Default to pink if not specified
        darker_color = [max(0, c - 0.2) for c in base_color]  # Reduce each RGB component
        node_colors[c] = darker_color if op in ('AND', 'OR', 'XOR') else base_color

    pos = nx.nx_agraph.graphviz_layout(G, prog="neato")

    nx.draw(G, pos, with_labels=True, node_color=[node_colors.get(node, 'pink') for node in G.nodes()], font_size=7, arrows=True, connectionstyle="arc3,rad=0.1")
    edge_labels = nx.get_edge_attributes(G, 'operation')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=1, label_pos=0.3, rotate=False)

    plt.title("Circuit Visualization")
    plt.show()

visualize_circuit(puzzle.input_data)

In [ ]:
def generate_n_bit_adder(n):
    """
    Generates the circuit description for an n-bit binary adder using the specified syntax.

    Args:
        n: The number of bits in the adder.
    """

    # Input declarations
    for i in range(n):
        print(f"x{i:02}: 0")
        print(f"y{i:02}: 0")
    print("c-1: 0")  # Initial carry-in

    # Full adder logic for each bit
    for i in range(n):
        # Previous carry-in
        if i == 0:
            prev_carry = "c-1"
        else:
            prev_carry = f"c{i-1:02}"

        # XOR gates for sum
        if i == 0:
          print(f"x{i:02} XOR y{i:02} -> i{i:02}") # First XOR
        else:
          print(f"x{i:02} XOR y{i:02} -> t{i:02}") # First XOR
          print(f"t{i:02} XOR {prev_carry} -> i{i:02}")

        print(f"i{i:02} XOR {prev_carry} -> z{i:02}")  # Final XOR for sum bit

        # AND and OR gates for carry-out
        print(f"x{i:02} AND y{i:02} -> a{i:02}")  # First AND
        if i == 0:
          print(f"x{i:02} XOR y{i:02} -> t{i:02}")
        print(f"t{i:02} AND {prev_carry} -> b{i:02}")  # Second AND
        print(f"a{i:02} OR b{i:02} -> c{i:02}")  # OR for carry-out

    # Final carry-out (extra bit)
    print(f"c{n-1:02} -> z{n:02}")

# Example: Generate a 4-bit adder
generate_n_bit_adder(4)

In [ ]:
def generate_n_bit_adder(n):
    """
    Generates the circuit description for an n-bit binary adder using the specified syntax.

    Args:
        n: The number of bits in the adder.
    """

    # Input declarations
    for i in range(n):
        print(f"x{i:02}: 0")
        print(f"y{i:02}: 0")

    # Full adder logic for each bit
    for i in range(n):
        # Previous carry-in
        if i == 0:
            prev_carry = None  # No carry-in for the least significant bit
        else:
            prev_carry = f"c{i-1:02}"

        # XOR gates for sum
        if i == 0:
          print(f"x{i:02} XOR y{i:02} -> z{i:02}") # First XOR, directly to sum
        else:
          print(f"x{i:02} XOR y{i:02} -> t{i:02}") # First XOR
          print(f"t{i:02} XOR {prev_carry} -> z{i:02}")

        # AND and OR gates for carry-out (only if not the last bit)
        if i < n - 1:
          if i == 0:
            print(f"x{i:02} AND y{i:02} -> c{i:02}")  # Only AND for carry-out
          else:
            print(f"x{i:02} AND y{i:02} -> a{i:02}")  # First AND
            print(f"t{i:02} AND {prev_carry} -> b{i:02}")  # Second AND
            print(f"a{i:02} OR b{i:02} -> c{i:02}")  # OR for carry-out
        elif i == n-1:
          print(f"x{i:02} AND y{i:02} -> a{i:02}")  # First AND
          print(f"t{i:02} AND {prev_carry} -> b{i:02}")  # Second AND
          print(f"a{i:02} OR b{i:02} -> z{i+1:02}")

    # Final carry-out (extra bit) - now handled in the loop

# Example: Generate a 4-bit adder
generate_n_bit_adder(45)

In [ ]:

print_graph_difference(input_a, input_b)

In [ ]:
def s(w, v):
    w.update(v)

def e(g, w):
    o = []
    for a, op, b, _, c in g:
        if a in w and b in w:
            if op == 'AND':
                w[c] = w[a] & w[b]
            elif op == 'OR':
                w[c] = w[a] | w[b]
            elif op == 'XOR':
                w[c] = w[a] ^ w[b]
            o.append((a, op, b, _, c))
    for x in o:
        g.remove(x)

def simulate_circuit(input_data, a, b):
    """
    Simulates the circuit described in the input data with given input values.

    Args:
        input_data: The raw puzzle input string.
        a: The integer value to assign to the 'x' inputs.
        b: The integer value to assign to the 'y' inputs.

    Returns:
        The integer represented by the 'z' output wires after simulation.
    """
    i, g = input_data.split('\n\n')
    w_init = {}
    for l in i.splitlines():
        n, v = l.split(': ')
        w_init[n] = int(v)
    g = [tuple(l.split()) for l in g.splitlines()]

    x_bits = len([k for k in w_init if k.startswith('x')])
    y_bits = len([k for k in w_init if k.startswith('y')])

    if a >= 2**x_bits or b >= 2**y_bits:
        raise ValueError("Input values are too large for the number of x/y bits.")

    w = w_init.copy()

    # Assign input values
    for i in range(x_bits):
        w[f'x{i:02}'] = (a >> i) & 1
    for i in range(y_bits):
        w[f'y{i:02}'] = (b >> i) & 1

    # Simulate the circuit
    while g:
        e(g, w)

    # Extract output values
    z_bits = len([k for k in w if k.startswith('z')])
    z_val = 0
    for i in range(z_bits):
        z_val |= (w.get(f'z{i:02}', 0) << i)

    return z_val

# Example usage:
input_data = """
x00: 0
x01: 0
x02: 0
y00: 0
y01: 0
y02: 0

x00 XOR y00 -> z00
x01 XOR y01 -> t01
x00 AND y00 -> c00
t01 XOR c00 -> z01
x02 XOR y02 -> t02
x01 AND y01 -> a01
t01 AND c00 -> b01
a01 OR b01 -> c01
t02 XOR c01 -> z02
x02 AND y02 -> a02
t02 AND c01 -> b02
a02 OR b02 -> z03
"""
a = 5  # 101 in binary
b = 3  # 011 in binary
result = simulate_circuit(input_data, a, b)
print(f"For a={a}, b={b}, the result is: {result}")  # Expected output: 8

In [75]:
fixed_input = puzzle.input_data

# TBD - swap the known-need-to-be-swapped wires.

```
x = x08
y = y08
s1 = mcr
cin = sjd
z = z08
c1 = wdc
c2 = mvb

x08 XOR y08 -> mcr (s1)
sjd XOR mcr -> mvb (should be z08)
x08 AND y08 -> wdc (c1)
mcr AND sjd -> z08 (should be c2)
qtg OR qtw -> sjd (c08)
mvb OR wdc -> ggm (cout)

```

Therefore mvb and z08 are swapped.

Second anomoly z14
```
14 0b1000000000000000
15 0b1000000000000000
```

```
model:

x XOR y -> s1
x AND y -> c1
s1 XOR cin -> z
s1 AND cin -> c2
c1 OR c2 -> cout

actual:

x14 XOR y14 -> jss <-- must be s1
x14 AND y14 -> rds
scs XOR rds -> z14
scs AND rds -> dcv <-- must be c2
dcv OR jss -> tsg <-- jss must be either c1 or c2, can't be c2, so must be c1

therefore s1 and c1 are swapped
therefore rds and jss are swapped

map

x = x14
y = y14
z = z14
s1 = jss or (--scs-- or rds)
cin = scs
c1 = rds or jss
c2 = dcv
cout = 
```

18 0b10000000000000000000
19 0b10000000000000000000

```
model:

x XOR y -> s1
x AND y -> c1
s1 XOR cin -> z
s1 AND cin -> c2
c1 OR c2 -> cout

actual:
x18 XOR y18 -> fmm <-- s1
x18 AND y18 -> z18 <-- should be wss c1
mfk XOR fmm -> wss <-- should be z18 (z)
mfk AND fmm -> shw <-- c2
wss OR shw -> nws <-- cout

Therefore, swap z18 and wss
```

```
23 0b1000000000000000000000000
24 0b1000000000000000000000000
```
```
model:

x XOR y -> s1
x AND y -> c1
s1 XOR cin -> z
s1 AND cin -> c2
c1 OR c2 -> cout

actual:
x23 XOR y23 -> qmd <- s1
x23 AND y23 -> fwj <- c1
qmd XOR bpr -> bmn <- should be z23
qmd AND bpr -> vsq <- c2
fwj OR vsq -> z23 <-- that's wrong, should be cout

So swap z23 and bmn

```

In [76]:
_, gate_connections = fixed_input.split('\n\n')
circuit = {}
for line in gate_connections.splitlines():
    inputs, output = line.split(' -> ')
    input1, op, input2 = inputs.split()
    circuit[output] = (op, input1, input2)


In [56]:
def eval_circuit(circuit, x, y, x_bits, y_bits, z_bits):
    wires = {}
    xb = bin(x)[2:].zfill(x_bits)
    yb = bin(y)[2:].zfill(y_bits)

    # Assign bits in reverse order (least significant bit first)
    for i in range(x_bits):
        wires[f'x{i:02}'] = int(xb[x_bits - 1 - i])
    for i in range(y_bits):
        wires[f'y{i:02}'] = int(yb[y_bits - 1 - i])
    
    while True:
        updated = False
        for output, (op, input1, input2) in circuit.items():
            if input1 in wires and input2 in wires and output not in wires:
                val1 = wires[input1]
                val2 = wires[input2]
                if op == 'AND':
                    wires[output] = val1 & val2
                elif op == 'OR':
                    wires[output] = val1 | val2
                elif op == 'XOR':
                    wires[output] = val1 ^ val2
                updated = True
        if not updated:
            break
    
    # Concatenate z_wires in reverse order for correct binary representation
    z_wires = sorted([wire for wire in wires if wire.startswith('z')], reverse=True)
    binary_string = ''.join(str(wires[wire]) for wire in z_wires[len(z_wires) - z_bits:])

    return int(binary_string, 2)


In [61]:
def e(x,y):
    return eval_circuit(circuit, x, y, 45, 45, 46)

In [77]:
for i in range(45):
    print(i, bin(e(1 << i, 0)))

0 0b1
1 0b10
2 0b100
3 0b1000
4 0b10000
5 0b100000
6 0b1000000
7 0b10000000
8 0b100000000
9 0b1000000000
10 0b10000000000
11 0b100000000000
12 0b1000000000000
13 0b10000000000000
14 0b100000000000000
15 0b1000000000000000
16 0b10000000000000000
17 0b100000000000000000
18 0b1000000000000000000
19 0b10000000000000000000
20 0b100000000000000000000
21 0b1000000000000000000000
22 0b10000000000000000000000
23 0b1000000000000000000000000
24 0b1000000000000000000000000
25 0b10000000000000000000000000
26 0b100000000000000000000000000
27 0b1000000000000000000000000000
28 0b10000000000000000000000000000
29 0b100000000000000000000000000000
30 0b1000000000000000000000000000000
31 0b10000000000000000000000000000000
32 0b100000000000000000000000000000000
33 0b1000000000000000000000000000000000
34 0b10000000000000000000000000000000000
35 0b100000000000000000000000000000000000
36 0b1000000000000000000000000000000000000
37 0b10000000000000000000000000000000000000
38 0b10000000000000000000000000000000000

In [80]:
swapped = ['mvb','z08','rds','jss','z18','wss','z23','bmn']
puzzle.answer_b = ','.join(sorted(swapped))

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 24! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
